# Supervised Learning: Classifiers

In the last lecture, we talked about classifiers.  In this lecture, we will actually _use_ the classifiers!

Recall that classifiers accept a sample value, then determine if the sample belongs to _class 1_ or _class 2_.  It makes this decision based on a training set which included not only samples, but for each sample an indication of which class that sample belonged to.

The classifier may not (i.e. probably will not) be perfect, it may have some errors.  The correct responses include:

* True Positives: samples that the classifier said should be in _class 1_, and the sample indeed belongs in _class 1_.

* True Negatives: samples that the classifier said should be in _class 2_, and the sample indeed belongs in _class 2_.

The incorrect responses are:

* False Positives: samples that the classifier said should be in _class 1_, but actually the sample belongs in _class 2_.

* False Negatives: samples that the classifier said should be in _class 2_, but actually the sample belongs in _class 1_.

Naturally, we want the classifier to always compute the correct response, so that the False Positives and False Negatives never occur.

## The Problem

For this chapter, the book uses data from the Lending Club, which is a peer-to-peer lending company.  Some members of the company are looking for loans, and other members of the company are looking to loan money.  A client (potentially a borrower) submits an application for a loan.  The company assesses the risk, and may decide to accept or reject the application.  If it is accepted, then an investor (potentially a lender) may decide to loan the full amount, or perhaps a partial amount.  There may be multiple investors funding the loan.

The question we want to answer is, "Given the data on the application, will this loan be fully funded?"  This is a classification problem, the answer will be either 'Yes' or 'No'.  Since the answer has only two possible values, this is a _binary_ classifier.  If there were more that two possible answers, this would be a _multiclass_ classifier.

In a few moments, we will start poking into the data, but first we will set up the Notebook.  For now, we will load the following packages.  Later we might add some additional packages.

* pickle -- this is used to read and write Python objects to binary data files.  This is similar to JSON and Javascript, but the file format is not human-readable.

* several tools from scikit-learn -- neighbors (to get the KNN classifier), datasets, metrics.

* matplotlib -- to draw graphs

* numpy -- numeric processing procedures and structures

In [1]:
import pickle
from sklearn import neighbors
from sklearn import datasets
from sklearn import metrics
#%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np

ModuleNotFoundError: No module named 'sklearn'

Things have changed a lot in this ecosystem since the book was written.  So some of the packages we use are written to use code that will at some point soon will be deprecated.  So we will see several warning messages.  We can simply ignore these messages for now, or we can modify the code to not use those features.  But the code which uses these features are some of these packages that we have imported.  We cannot upgrade the code, we are waiting for the authors of those packages to do this.

So for now, we just want to turn off the message.  The following code will turn off all warning messages.  Not the best practice, but I've looked at the messages, found they don't apply right now, so then I came back here and turned off the warnings.

In [ ]:
import warnings 
warnings.filterwarnings("ignore")

This next cell loads the data from the pkl file.  The data consists of two arrays:

* The first array, which we will call 'x', is a 2-dimensional array.  Each row in the array is one loan application, and each column is one of the attributes from the application.  We don't know what each of the columns represent, but for this application we don't need to know!

* The second array, which we will call 'y', is a 1-dimensional array indicating whether the application was fully funded (1.0) or not fully funded (-1.0).

In [ ]:
ofname = open('dataset_small.pkl', 'rb')
(x, y) = pickle.load(ofname, encoding="latin1")

We can take a peek at the x array:

In [ ]:
x

... and the y array:

In [ ]:
y

These arrays are Numpy arrays.

We don't know the sizes of the arrays, but this would be good to know.  We can ask for an array's _shape_, which gives us the number of entries in the array.

Since _x_ is a 2-dimensional array, _shape_ will return an array, 2 long, giving the counts for the two dimensions.  In the following code, we separate and then print out these two values.

In [ ]:
dims = x.shape[1]
N = x.shape[0]
print(f'dims: {dims}, samples: {N}')

## KNN Classification

Let's start by making a K-Nearest Neighbor classifier.  We will arbitrarily use _k = 11_, so when we ask for a prediction, it will find the 11 nearest neighbors, then merge the results to come up with a final result.

In the following, we create the classifier, then train it with the data we loaded above.  We then do a prediction _with the same data we used to train the classifier_.  We will then look at the last sample, just for example, to see what the actual and what the predicted results are.  Ideally, these would have the same value!

In [ ]:
# Create an instance of K-nearest neighbor classifier
knn = neighbors.KNeighborsClassifier(n_neighbors = 11)

# Train the classifier
knn.fit(x, y)

# Compute the prediction according to the model
yhat = knn.predict(x)

# Check the results on the last sample
print(f'Predicted value: {yhat[-1]}, real target: {y[-1]}')

Great!  We predicted that the last application would not be funded, and it wasn't!  What about all of the other cases?

Classifiers have a method, _score_, which is passed an array of applications (input samples) and a vector of expected values (output results).  The method will do a _predict_, then compute the percentage of predictions actually match the expected values.

We will pass the original data, both the applications and results, to score.  So the same data we used to train the classifier is being used to test the classifier.  Ideally, our score should be 100%

In [ ]:
knn.score(x, y)

Why is the score not 100%?

We are looking at the 11 nearest neighbors, then computing some form of averaging of the result.  One of the nearest neighbors will in fact be the input sample.  However, we are 'blurring' the results with the other 10 values.  Perhaps enough of the other neighbors have the opposite value, so then the classifier will return the wrong result.  Evidently, in about 17% of the cases, this is what is happening!

In a bit we will try tuning the classifier, and hopefully we will get a better score.

But in the mean time, the book wanted to explore the data a little more.  The question they want to answer is, "What percentage of applications are funded?"  They built a pie chart showing the answer:

In [ ]:
plt.pie(np.c_[np.sum(np.where(y==1,1,0)),np.sum(np.where(y==-1,1,0))][0],
        labels=['Not fully funded','Full amount'],
        colors=['g','r'],
        shadow=False,
        autopct ='%.2f' )
plt.gcf().set_size_inches((6,6))

Interestingly enough, about 82% of the applications were funded.  Our classifier had about 83% accuracy.

If we built a classifier that just said that every application was fully funded, we would be right about 82% of the time, which is just about as good as our fancy classifier!  That's not very encouraging!  (And it fact, it is a little coincidental that the numbers were so similar.)

## Confusion Matrix

We might glean additional information, and perhaps some insights in improving the situation, by looking closer at what we were getting wrong.  The first step of doing this is to consider the _confusion matrix_.  Recall that this matrix displays four values:

* How many times did the prediction say the answer was 'Yes' when the actual value was 'Yes'?  These cases are correct predictions, the _True Positives_.

* How many times did the prediction say the answer was 'No' when the actual value was 'No'?  These are also correct predictions, the _True Negatives_.

* How many times did the prediction say the answer was 'Yes', when the actual value was 'No'?  These are incorrect predictions, the _False Positives_.

* How many times did the prediction say the answer was 'No', when the actual value was 'Yes'?  These are incorrect predictions, the _False Negatives_.

These values are easy to compute:

In [ ]:
TP = np.sum(np.logical_and(yhat == -1, y == -1))
TN = np.sum(np.logical_and(yhat == 1, y == 1))
FP = np.sum(np.logical_and(yhat == -1, y == 1))
FN = np.sum(np.logical_and(yhat == 1, y == -1))
print(f'TP: {TP:4}, FP: {FP:4}')
print(f'FN: {FN:4}, TN: {TN:4}')

While this code was pretty simple, the confusion matrix is a common metric, so they have actually built a function which computes this matrix:

In [ ]:
metrics.confusion_matrix(yhat, y)

Recall that right now, we are running the prediction on exactly the same data as we used to train the model, so why is the score so low?  Looking at the matrix, the major problem is the false positives: in too many cases, the prediction thinks the answer should be 'Yes', when really it should be 'No'.

Recall how the _KNN_ classifier works:

* It keeps the list of training samples, along with their output values.

* When a sample is being predicted, it finds a number of these training vectors, the _k_ closest ones.  It then performs some type of averaging (different implementations use different averaging techniques), to pick an answer.

In too many cases, when we are gathering the list of nearest neighbors, we gather too many that belong to the other set, the wrong answer.

Remember also how skewed the data set is: for every 'No' answer, there are four 'Yes' answers.  So when we find neighbors around a 'No' sample, there is a good chance that the majority of these neighbors will belong to the 'Yes' state.

So let's vary the number of neighbors.  Perhaps we will get better results with fewer neighbors confusing the issue!

I took the code from the book, and wrote it as a function, so that it is easier to run a number of cases.  And in fact, I split it into two functions, one to perform the calculation and the other to print the results.  The reason for that is because some of our examples later on will use the same reporting function.

In [ ]:
# Vary the number of neighbors in the classifier

# This prints the statistics, comparing the result (output of the predictor) with
# the answer (the actual values for those samples)
def classifierStats(label,result,answer):
    print(f'--- {label} Stats ---')
    print(f'classification accuracy: {100*metrics.accuracy_score(result, answer):.2f}%')
    print(f'confusion matrix:\n{metrics.confusion_matrix(result, answer)}')

# Given a dataset (x) and the actual outputs (y), build a KNN with a given number
# of nearest neighbors (num_neigh), then print the statistics
def classifyWithNeighbors(x, y, num_neigh):
    knn = neighbors.KNeighborsClassifier(n_neighbors = num_neigh)
    knn.fit(x, y)
    yhat = knn.predict(x)
    classifierStats(f'{num_neigh} Nearest Neighbors', yhat, y)
    
classifyWithNeighbors(x, y, 1)

Wow!  This is great results!

Not really, unfortunately.  The samples we are trying to predict are exactly the same samples we used to train.  And since we are only asking for the one nearest neighbor, we are going to find in each case the same sample in the training set, so of course the answer is correct.

However, we will see later how a KNN with _K = 1_ is still a valid option for a classifier.

We can make a loop to try several different sizes, to see if we can find a pattern.  One thing they said about KNN, it is best to have an odd number of neighbors (to break ties).

In [ ]:
for i in range(1, 11, 2):
    classifyWithNeighbors(x, y, i)

Here we see that smaller neighborhood sizes work better.  For other problems, with other data, we may find different results.  Sometimes larger numbers of neighbors works better!

## Sampling

As we stated above, a big flaw with the approach we have used so far is that we are doing a prediction on exactly the same data we used for training!  So we don't really know how well our classifier is working.  We want the classifier to work well with _new_ data, data we haven't yet seen.

Actually, we can solve this problem by splitting the dataset into two parts: the training part and the testing part.  The training part will be used to train (fit) the classifier.  We can then run the samples of the testing part through the predictor, then compare the results with the known answers for each of those testing samples.  This will tell us how well the classifier works with data _not_ used in the training.

We will start by permuting the samples.  Why?  Perhaps the input data is sorted in some manner.  For example, maybe it is sorted by date, and maybe the results vary based on time of year.  If so, then if we just split the dataset, we might be training with 'early in the year' data, but then testing with 'late in the year' data.  So the training would be skewed to correctly predict early in the year samples.

Numpy has a method for computing a permutation matrix:

In [ ]:
# Make a permutation list
perm = np.random.permutation(10)
perm

The following makes the actual permutation list we will use, the size matches the size of our data.

The next thing we need is the _split point_: at what index do we divide between the training set and the testing set.  We want a given percentage of data in the training set, so we compute the index that is that percentage of the size of the data.

In [ ]:
# Randomize the data, just in case the data was coming in some sort of order

perm = np.random.permutation(y.size)

# PRC gives the percentage of values that we want in the training set
PRC = 0.7
split_point = int(np.ceil(y.shape[0] * PRC))
split_point

To build the training set, we want to use all of the entries in the permutation list, from 0 up to the split point, to index into the dataset.

To build the test set, we want the portion of the permutation list, from the split point to the end.

In [ ]:
# Generate the training and test sets
X_train = x[perm[:split_point], : ]
Y_train = y[perm[:split_point]]
X_test = x[perm[split_point:], : ]
Y_test = y[perm[split_point:]]

Let's now look at the shape of the data, to verify that our splitting worked:

In [ ]:
print(f'X training shape: {X_train.shape}, Y training shape: {Y_train.shape}')
print(f'X test shape: {X_test.shape}, Y test shape: {Y_test.shape}')

We can now train the classifier with the training set...

In [ ]:
# Train a classifier
knn = neighbors.KNeighborsClassifier(n_neighbors = 1)
knn.fit(X_train, Y_train)
yhat = knn.predict(X_train)
classifierStats('Training', yhat, Y_train)

...then evaluate the prediction using the testing data:

In [ ]:
# Check the test case
yhat = knn.predict(X_test)
classifierStats('Testing', yhat, Y_test)

Well, that's not so great!  With _K = 1_, the KNN classifier didn't do so well with unseen samples (the testing samples vs the training samples).  Let's explore varying the neighborhood size:

In [ ]:
NUM_RUNS = 20
scores = []
xvals = list(range(1, 2 * NUM_RUNS, 2));
for size in xvals:
    knn = neighbors.KNeighborsClassifier(n_neighbors = size)
    knn.fit(X_train, Y_train)
    yhat = knn.predict(X_test)
    scores.append(100 * metrics.accuracy_score(yhat, Y_test))

fig, ax = plt.subplots(1, 1, figsize=(12, 4))
plt.plot(xvals, scores);
plt.ylabel('Accuracy')
plt.xlabel('Neighborhood size')
plt.xticks(xvals)
plt.show()

As we can see, for this data, KNN did a really poor job for _K = 1_, while when just using the testing data, that wasn't such a good idea.  But we get good results for size 5 and above (again, we prefer an odd size).

## Sample Distributions

What we saw was the results of running one sample.  This sample was randomly chosen.  If we randomly choose another sample, this will have different statistics.

So why don't we run a number of tests, then look at the distribution of the scores?

After showing us the tedious way to split our dataset into test and training subsets, the book then went on to say that scikit-learn has a method for doing this split: _train_test_split_.  We will use this function for our test (but we have to import that function).  One thing to note is that in our old code, the percentage we used was the percentage to put in the training set.  For this new method, the percent is the amount to put in the testing set!  So we will change the _PRC_ value accordingly.

The other thing that is a bit weird is that they called the result 'Mean expected error', but really, this is the 'Mean expected accuracy'.  Not sure why they did that!  So I changed the label!

Finally, one other thing: they ran the test with _K = 1_ which we know is pretty poor.  So I switched to using _K = 11_, so the numbers here look better than what the book shows.

In [ ]:
from sklearn.model_selection import train_test_split
PRC = 0.3
num_tests = 10
acc = np.zeros((num_tests,))
for i in range(num_tests):
    X_train, X_test, Y_train, Y_test = train_test_split(x, y, test_size = PRC)
    knn = neighbors.KNeighborsClassifier(n_neighbors = 9)
    knn.fit(X_train, Y_train)
    yhat = knn.predict(X_test)
    acc[i] = 100 * metrics.accuracy_score(yhat, Y_test)
acc.shape = (1, num_tests)
print(f'Mean expected accuracy: {np.mean(acc[0]):.2f}%')

__Tiny Digression__

In the above examples, we were using a 70/30 split for the training and testing set sizes.  In the next example in the book, they switched to a 90/10 split.  I was just curious, how does that affect the accuracy?  So I ran the following test:

In [ ]:
num_percents = 10
num_tests = 20
results = []
xvals = []
for j in range(num_percents):
    PRC = 0.5 * (j + 1) / num_percents
    xvals.append(PRC)
    acc = np.zeros((num_tests,))
    for i in range(num_tests):
        X_train, X_test, Y_train, Y_test = train_test_split(x, y, test_size = PRC)
        knn = neighbors.KNeighborsClassifier(n_neighbors = 9)
        knn.fit(X_train, Y_train)
        yhat = knn.predict(X_test)
        acc[i] = 100 * metrics.accuracy_score(yhat, Y_test)
    acc.shape = (1, num_tests)
    results.append(np.mean(acc[0]))
fig, ax = plt.subplots(1, 1, figsize=(12, 4))
plt.plot(xvals, results);
plt.ylabel('Mean Expected Accuracy')
plt.xlabel('Test Size (as percent of whole set)')
plt.xticks(xvals)
plt.show()

I actually ran this test several times, and even tried varying the number of tests.  The graph came out looking completely different each time.  On the other hand, all of these values are within one percent.  So it seems like the percent pulled aside didn't matter that much.  However, I think for the really small percentages (<10%), there was a lot more variability, probably because there just weren't enough datapoints.

## Other Classifiers

For the next investigation, the book added additional classifiers, specifically the _Support Vector Machines_ (SVM) and _Decision Trees_.  After importing these packages, these new classifiers can be dropped in (although tuning parameters may improve the results).  Each of the classifiers has the same _fit_ and _predict_ interface.

One change: they used _K = 1_ and _K = 3_ for the KNN classifier.  I switched to use _K = 5_ and _K = 11_.

In [ ]:
from sklearn import tree
from sklearn import svm

PRC = 0.1
acc_r = np.zeros((10,4))
for i in range(10):
    X_train, X_test, y_train, y_test = train_test_split(x, y, test_size = PRC)
    nnA = neighbors.KNeighborsClassifier(n_neighbors = 5)
    nnB = neighbors.KNeighborsClassifier(n_neighbors = 11)
    svc = svm.SVC()
    dt = tree.DecisionTreeClassifier()
    
    nnA.fit(X_train,y_train)
    nnB.fit(X_train,y_train)
    svc.fit(X_train,y_train)
    dt.fit(X_train,y_train)
    
    yhat_nnA = nnA.predict(X_test)
    yhat_nnB = nnB.predict(X_test)
    yhat_svc = svc.predict(X_test)
    yhat_dt = dt.predict(X_test)
    
    acc_r[i][0] = metrics.accuracy_score(yhat_nnA, y_test)
    acc_r[i][1] = metrics.accuracy_score(yhat_nnB, y_test)
    acc_r[i][2] = metrics.accuracy_score(yhat_svc, y_test)
    acc_r[i][3] = metrics.accuracy_score(yhat_dt, y_test)


plt.boxplot(acc_r);
for i in range(4):
    xderiv = (i+1)*np.ones(acc_r[:,i].shape)+(np.random.rand(10,)-0.5)*0.1
    plt.plot(xderiv,acc_r[:,i],'ro',alpha=0.3)
    
ax = plt.gca()
ax.set_xticklabels(['5-NN','11-NN','SVM','Decision Tree'])
plt.ylabel('Accuracy')
plt.show()

For this particular data set and for this example, it looks like KNN(11) is our best bet, but SVN is very close.

## Learning Curves

The book next investigated the effect of number of examples on the test and training errors, and they used a Decision Tree as the classifier.

They also used a new dataset, a Toy problem, built as follows:

In [ ]:
%matplotlib inline
MAXN = 700

fig = plt.figure()
fig.set_size_inches(6,5)

plt.plot(1.25*np.random.randn(MAXN,1),1.25*np.random.randn(MAXN,1),'r.',alpha = 0.3)
plt.plot(8+1.5*np.random.randn(MAXN,2),5+1.5*np.random.randn(MAXN,2),'r.', alpha = 0.3)
plt.plot(5+1.5*np.random.randn(MAXN,1),5+1.5*np.random.randn(MAXN,1),'b.',alpha = 0.3)
plt.show()

They generated the above plot to be representative of the data, but they generate different datasets below.  They will run 10 passes through the system, each randomly generated.  But all of the samples have the same general form.

One parameter they can change is the _complexity_ of the Decision Tree.  The complexity is related to the depth of the tree.  One tuning parameter for the Decision Tree is the maximum depth.  By limiting this depth, the tree might have to stop segregating the values early, so there are fewer tests during evaluation, but there is some error due to less segregating of the data.

We'll see the graph first, then we'll discuss how the graph was built.

In [ ]:
import numpy as np
from sklearn import metrics
from sklearn import tree

C=5
MAXN=1000

yhat_test=np.zeros((10,299,2))
yhat_train=np.zeros((10,299,2))
#Repeat ten times to get smooth curves
for i in range(10):
    X = np.concatenate([1.25*np.random.randn(MAXN,2),5+1.5*np.random.randn(MAXN,2)]) 
    X = np.concatenate([X,[8,5]+1.5*np.random.randn(MAXN,2)])
    y = np.concatenate([np.ones((MAXN,1)),-np.ones((MAXN,1))])
    y = np.concatenate([y,np.ones((MAXN,1))])
    perm = np.random.permutation(y.size)
    X = X[perm,:]
    y = y[perm]

    X_test = np.concatenate([1.25*np.random.randn(MAXN,2),5+1.5*np.random.randn(MAXN,2)]) 
    X_test = np.concatenate([X_test,[8,5]+1.5*np.random.randn(MAXN,2)])
    y_test = np.concatenate([np.ones((MAXN,1)),-np.ones((MAXN,1))])
    y_test = np.concatenate([y_test,np.ones((MAXN,1))])
    j=0
    for N in range(10,3000,10):
        Xr=X[:N,:]
        yr=y[:N]
        #Evaluate the model
        clf = tree.DecisionTreeClassifier(min_samples_leaf=1, max_depth=C)
        clf.fit(Xr,yr.ravel())
        yhat_test[i,j,0] = 1. - metrics.accuracy_score(clf.predict(X_test), y_test.ravel())
        yhat_train[i,j,0] = 1. - metrics.accuracy_score(clf.predict(Xr), yr.ravel())
        j=j+1

p1,=plt.plot(np.mean(yhat_test[:,:,0].T,axis=1),'pink')
p2,=plt.plot(np.mean(yhat_train[:,:,0].T,axis=1),'c')
fig = plt.gcf()
fig.set_size_inches(12,5)
plt.xlabel('Number of samples x10')
plt.ylabel('Error rate')
plt.legend([p1,p2],["Test C = 5","Train C = 5"])
plt.show()

They started by making a vector of training data and a vector of test data.  Each of these sets consisted of 1000 random points centered around 0,0 (belonging to the 'Yes' group), 1000 points centered around 5,5 (belonging to the 'No' group), and 1000 points centered around 8,5 (belonging to the 'Yes' group).  They then permuted these arrays.

Next, they ran a series of evaluations.  For each, they selected _n_ of the training points, but all of the test points.  _n_ varied from 10 to 3000 in steps of 10.  So for the first several evaluations, they had very little training data!

The lower curve of the graph shows the error rate for predicting the training data, and the upper curve is the test data.

Note for really small sample sizes, the classifier did a great job on the training data.  Consider the problem!  There are three clusters of points.  All of the points centered around 0,0 are clearing in group 'Yes', so it is not problem identifying them.  But the other two clusters partially overlap.  In the heart of the overlap region, how can the classifier correctly separate essentially identical values.  It looks like there is a built-in error rate of about 10% due to this region of overlapping points.

* As the number of samples increases, both curves tend towards the same value.

* When we have few training data, the training prediction has a low error rate, but the test predition has a high error rate.

Next they repeated the test with a less-complex tree, the depth is limited to 1 (the previous was 5).

In [ ]:
C=1
MAXN=1000

#Repeat ten times to get smooth curves
for i in range(10):
    X = np.concatenate([1.25*np.random.randn(MAXN,2),5+1.5*np.random.randn(MAXN,2)]) 
    X = np.concatenate([X,[8,5]+1.5*np.random.randn(MAXN,2)])
    y = np.concatenate([np.ones((MAXN,1)),-np.ones((MAXN,1))])
    y = np.concatenate([y,np.ones((MAXN,1))])
    perm = np.random.permutation(y.size)
    X = X[perm,:]
    y = y[perm]

    X_test = np.concatenate([1.25*np.random.randn(MAXN,2),5+1.5*np.random.randn(MAXN,2)]) 
    X_test = np.concatenate([X_test,[8,5]+1.5*np.random.randn(MAXN,2)])
    y_test = np.concatenate([np.ones((MAXN,1)),-np.ones((MAXN,1))])
    y_test = np.concatenate([y_test,np.ones((MAXN,1))])
    j=0
    for N in range(10,3000,10):
        Xr=X[:N,:]
        yr=y[:N]
        #Evaluate the model
        clf = tree.DecisionTreeClassifier(min_samples_leaf=1, max_depth=C)
        clf.fit(Xr,yr.ravel())
        yhat_test[i,j,1] = 1. - metrics.accuracy_score(clf.predict(X_test), y_test.ravel())
        yhat_train[i,j,1] = 1. - metrics.accuracy_score(clf.predict(Xr), yr.ravel())
        j=j+1

p3,=plt.plot(np.mean(yhat_test[:,:,1].T,axis=1),'r')
p4,=plt.plot(np.mean(yhat_train[:,:,1].T,axis=1),'b')
fig = plt.gcf()
fig.set_size_inches(12,5)
plt.xlabel('Number of samples x10')
plt.ylabel('Error rate')
plt.legend([p3,p4],["Test C = 1","Train C = 1"])
plt.show();

With this simpler classifier, the system 'settled' much quicker, but at a much higher error rate, around 33%.  Clearly the trees did not have enough depth to adequately distinguish the samples.

In this next graph, we show both on the same graph, to more clearly show the difference.

In [ ]:
p1,=plt.plot(np.mean(yhat_test[:,:,0].T,axis=1),color='pink')
p2,=plt.plot(np.mean(yhat_train[:,:,0].T,axis=1),'c')
p3,=plt.plot(np.mean(yhat_test[:,:,1].T,axis=1),'r')
p4,=plt.plot(np.mean(yhat_train[:,:,1].T,axis=1),'b')
fig = plt.gcf()
fig.set_size_inches(12,5)
plt.xlabel('Number of samples x10')
plt.ylabel('Error rate')
plt.legend([p1,p2,p3,p4],["Test C = 5","Train C = 5","Test C = 1","Train C = 1"])
fig = plt.gcf()
fig.set_size_inches(12,5)
plt.show()

* With a low degree of complexity, the training and test errors converge to the bias sooner, with fewer data.

* With a low degree of complexity, the error of convergence is larger than with increased complexity.

The value that both curves converge toward is called the _bias_, and the difference between this value and the test error is called the _variance_.

The previous tests were varying the number of samples.  In the next test, they use a fixed number of samples, but vary the complexity of the model:

In [ ]:
%reset -f
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
from sklearn import metrics
from sklearn import tree

MAXC=20
N=1000
NTEST=4000
ITERS=3

yhat_test=np.zeros((ITERS,MAXC,2))
yhat_train=np.zeros((ITERS,MAXC,2))
#Repeat ten times to get smooth curves
for i in range(ITERS):
    X = np.concatenate([1.25*np.random.randn(N,2),5+1.5*np.random.randn(N,2)]) 
    X = np.concatenate([X,[8,5]+1.5*np.random.randn(N,2)])
    y = np.concatenate([np.ones((N,1)),-np.ones((N,1))])
    y = np.concatenate([y,np.ones((N,1))])
    perm = np.random.permutation(y.size)
    X = X[perm,:]
    y = y[perm]

    X_test = np.concatenate([1.25*np.random.randn(NTEST,2),5+1.5*np.random.randn(NTEST,2)]) 
    X_test = np.concatenate([X_test,[8,5]+1.5*np.random.randn(NTEST,2)])
    y_test = np.concatenate([np.ones((NTEST,1)),-np.ones((NTEST,1))])
    y_test = np.concatenate([y_test,np.ones((NTEST,1))])

    j=0
    for C in range(1,MAXC+1):
        #Evaluate the model
        clf = tree.DecisionTreeClassifier(min_samples_leaf=1, max_depth=C)
        clf.fit(X,y.ravel())
        yhat_test[i,j,0] = 1. - metrics.accuracy_score(clf.predict(X_test), y_test.ravel())
        yhat_train[i,j,0] = 1. - metrics.accuracy_score(clf.predict(X), y.ravel())
        j=j+1

p1, = plt.plot(np.mean(yhat_test[:,:,0].T,axis=1),'r')
p2, = plt.plot(np.mean(yhat_train[:,:,0].T,axis=1),'b')
fig = plt.gcf()
fig.set_size_inches(12,5)
plt.xlabel('Complexity')
plt.ylabel('Error rate')
plt.legend([p1, p2], ["Testing error", "Training error"])
plt.show()

As we can see, as the complexity increases, the training error is reduced.  But above a certain level of complexity, the test error increases.  This effect is called _overfitting_.  Here are some techniques to overcome overfitting:

* Run an analysis like we did here, then choose the parameter value (in this case depth of the trees) that minimizes the error.  This is a form of model selection.

* Modify the model to penalize high-complexity. The book was a little confusing on this point!

* Use 'ensemble' techniques, such as the Random Forest, where we don't rely on a single classifier (Decision Tree in this case) to match all aspects of the data.

## Training, Validation, and Test

So far, we have been dividing our sample data into two sets, training and test.  With these two subsets, we can perform cross-validation to pick the best model to use for the data, or we can use cross-validation to tune a model (for example, selecting the best complexity for a Decision Tree).  But we can't do both, because these two optimizations would be correlated.

We could, instead, divide our sample data into three sets, _training_, _validation_, and _test_.

* The training data is used to select model from our list of possible models.

* The validation data is used to select the parameters of the model for the best performance.  This is a form of learning!

* The test data is used exclusively for assessing the performance at the end of the process and is never used in the learning process.